Данный файл реализует обучение сети на предсказание положения географического полюса земли.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib
import time
from matplotlib import pyplot
%run lstm.ipynb

## Гиперпараметры

In [ ]:
decimation = 30

num_epochs = 15
num_repeat = 1
dependant_coords = False # <= за формирование входной последовательности отвечают две сети

years = 7.5 # <= окно обучения 7.5 лет
output_dim = int(years * 365 / decimation)
input_dim = 2 * int(output_dim) if dependant_coords else int(output_dim)
hidden_dim = int(input_dim * 0.1)

learn_rate = 0.5
optimizer = 'adagrad'
do_normalize = True

do_svd = True
svd_iters = 10
svd_start = 0.0
svd_weaken = 0.15
svd_strength = svd_start

print('input', input_dim, 'hidden', hidden_dim, 'output', output_dim)

## Чтение данных

Пример чтения данных из таблицы в которой колонки разделены пробелами: https://stackoverflow.com/questions/43058462/reading-a-variable-white-space-delimited-table-in-python

Как объединить два массива с чередованием: https://stackoverflow.com/questions/3678869/pythonic-way-to-combine-two-lists-in-an-alternating-fashion

In [ ]:
path = 'e:/geomagnetic.txt'
data = np.genfromtxt(path)

# Достаём из файла 3 и 5 столбец где содержатся данные x и y.
# Децимируем данные в 8 раз иначе очень долгое врем яобучения.
x = data[:,3][::decimation]
y = data[:,5][::decimation]

assert len(x) == len(y)
print('Количество точек', len(x))

if do_normalize == True:
    m = max(np.amax(x), np.amax(y))
    x /= m
    y /= m
    
# Объединение с чередованием наборов точек x и y.
z = np.zeros(len(x) + len(y))
z[0::2] = x
z[1::2] = y

#matplotlib.pyplot.xlim(right=100)
#matplotlib.pyplot.plot(x)
#matplotlib.pyplot.plot(y)
#matplotlib.pyplot.plot(z)

## Подготовка данных для обучения

In [ ]:
def split(data, ratio):
    samples = int(len(data) * ratio)
    return data[:samples], data[samples:]

In [ ]:
def shift(data, dim):
    seq = []
    for i in range(len(data) - dim):
        seq.append(data[i:i+dim])
    return seq

In [ ]:
x_train, x_test = split(x, 0.8)
y_train, y_test = split(y, 0.8)
z_train, z_test = split(z, 0.8)

#x_train_in = shift(z_train, input_dim)
#y_train_in = shift(z_train, input_dim)
x_train_in = shift(x_train[0:], input_dim)
y_train_in = shift(y_train[0:], input_dim)

x_train_out = shift(x_train[1:], output_dim)
y_train_out = shift(y_train[1:], output_dim)

#matplotlib.pyplot.xlim(right=80)
#matplotlib.pyplot.plot(x_train)
#matplotlib.pyplot.plot(y_train)
#matplotlib.pyplot.plot(z_train)

#matplotlib.pyplot.plot(x_train_in[0])
#matplotlib.pyplot.plot(x_train_out[0])
#matplotlib.pyplot.plot(y_train_in[0])
#matplotlib.pyplot.plot(y_train_out[0])

## Обучение

In [ ]:
def train(lstm, data_in, data_out, num_epochs, num_repeat):
    time_begin = time.time()
    for e in range(num_epochs):
        lstm.clear_state()
        for rec_in, rec_out in zip(data_in, data_out):
            lstm.train([rec_in], [rec_out], num_repeat)
    return time.time() - time_begin

In [ ]:
def test(lstm, start, length, take=-1):
    
    points = []
    prev = [np.copy(start)]
    
    for i in range(length):
        
        prev = lstm.test(prev)
        
        points += [prev[-1][take]]
        
    return points

In [ ]:
# Параметр take описывает индекс массива который получается на комбинированном выходе двух сетей.
def test_dependant(x_lstm, y_lstm, start, length, take=-1):
    
    #assert len(start) == x_lstm.input_dim
        
    points = []
    prev = [np.copy(start)]
    
    for i in range(length):
        
        x_prev = x_lstm.test(prev)
        y_prev = y_lstm.test(prev)
                
        #assert len(x_prev[-1]) == x_lstm.output_dim
                
        # В силу особенностей форматы вводв/вывода сети, все данные помещаются в [].
        prev[-1][0::2] = x_prev[-1]
        prev[-1][1::2] = y_prev[-1]
        
        #assert len(prev[-1]) == x_lstm.input_dim
        
        points += [prev[-1][take]]
        
    return points

In [ ]:
def mae(x, y):
    assert len(x) == len(y)
    return np.sum(np.abs(x - y)) / len(x)

In [ ]:
def mse(x, y):
    assert len(x) == len(y)
    return np.sum(np.square(x - y)) / len(x)

In [ ]:
sess = tf.Session()
net_x = lstm_cell(sess, input_dim, hidden_dim, output_dim, learn_rate, optimizer)
net_y = lstm_cell(sess, input_dim, hidden_dim, output_dim, learn_rate, optimizer)

In [ ]:
x_time = []
y_time = []
x_mse = []
y_mse = []
x_mae = []
y_mae = []
x_svd = []
y_svd = []

In [ ]:
assert len(x_test) == len(y_test)

# При необходимости этот цикл можно повторить снова. Новая статистика будет добавлена в конец списка.
for s in range(svd_iters if do_svd else 1):
    
    x_time += [train(net_x, x_train_in, x_train_out, num_epochs, num_repeat)]
    y_time += [train(net_y, y_train_in, y_train_out, num_epochs, num_repeat)]
    
    net_x.save_all()
    net_y.save_all()
    
    if dependant_coords:
        x_predict = test_dependant(net_x, net_y, x_train_in[-1], len(x_test), -2)
        y_predict = test_dependant(net_x, net_y, y_train_in[-1], len(y_test), -1)
    else:
        x_predict = test(net_x, x_train_in[-1], len(x_test))
        y_predict = test(net_y, y_train_in[-1], len(y_test))
    
    net_x.restore_all()
    net_y.restore_all()
    
    x_mse += [mse(x_predict, x_test)]
    y_mse += [mse(y_predict, y_test)]
    
    x_mae += [mae(x_predict, x_test)]
    y_mae += [mae(y_predict, y_test)]
    
    if do_svd == True and svd_strength < 1.:
        
        x_svd += [net_x.svd_compress(svd_strength, svd_strength)[0]]
        y_svd += [net_y.svd_compress(svd_strength, svd_strength)[0]]
        
        svd_strength += svd_weaken
    
    print(s, end=' ')
    
net_x.restore_all()
net_y.restore_all()

## Статистика

In [ ]:
fig, ax = matplotlib.pyplot.subplots(2, 3, figsize=(10, 5))

ax[0,0].set_title('Обучение, сек.')
ax[0,0].plot(x_time)
ax[0,0].plot(y_time)

ax[0,1].set_title('Норма SVD')
ax[0,1].plot(x_svd)
ax[0,1].plot(y_svd)

ax[1,0].set_title('MAE')
ax[1,0].plot(x_mae)
ax[1,0].plot(y_mae)

ax[1,1].set_title('MSE')
ax[1,1].plot(x_mse)
ax[1,1].plot(y_mse)

ax[0,2].set_title('Предсказание X')
ax[0,2].plot(x_test)
ax[0,2].plot(x_predict, '--')

ax[1,2].set_title('Предсказание Y')
ax[1,2].plot(y_test)
ax[1,2].plot(y_predict, '--')

# Отображаем статистику обучения.
print('optimizer', optimizer, 'learn_rate', learn_rate)
print('input_dim', input_dim, 'hidden_dim', hidden_dim, 'output_dim', output_dim)
print('num_epochs', num_epochs, 'num_repeat', num_repeat)
print('years', years, 'dependant_coords', dependant_coords, 'do_normalize', do_normalize)
if do_svd:
    print('svd_start', svd_start, 'svd_end', svd_strength, 'svd_weaken', svd_weaken)
print('x_time', np.sum(x_time), 'y_time', np.sum(y_time))
print('x_mae', x_mae[-1], 'y_mae', y_mae[-1])
print('x_mse', x_mse[-1], 'y_mse', y_mse[-1])

In [ ]:
matplotlib.pyplot.plot(x_test, y_test)
matplotlib.pyplot.plot(x_predict, y_predict, '--')